In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 33.6 MB/s 
     |████████████████████████████████| 7.6 MB 64.1 MB/s 
     |████████████████████████████████| 182 kB 68.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 28.9 MB/s 
     |████████████████████████████████| 212 kB 71.7 MB/s 
     |████████████████████████████████| 132 kB 77.1 MB/s 
     |████████████████████████████████| 127 kB 69.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.6 MB/s 


In [11]:
#foldername= "/content/drive/My Drive/nlpproject/bertsequenceclassification/"
foldername= "/content/drive/My Drive/2022Fall/CSCI3349/NLP Project/extractedfeaturewithbert/"
#foldername= "/content/drive/My Drive/extractedfeaturewithbert/"


In [4]:
import numpy as np 
import pandas as pd 
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from datasets import load_metric
import datetime
from torch import nn
from transformers import AutoConfig
from transformers import AutoModel
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

#import keras
#from keras.models import Model
#import keras.backend as K
#from keras.callbacks import ModelCheckpoint
#from keras.models import load_model


In [5]:
class CFG():
  max_len = 50
  #model = "bert-base-uncased"
  model = "distilbert-base-uncased"
  lr=1e-4
  min_lr=1e-10
  batch_size=16
  epoch=1000
  embedding_dim=512
  hidden_dim=256
  drop=0.15
cfg=CFG()

In [6]:
class Ds(Dataset):
    def __init__(self, path, tokenizer, i, max_token_len=cfg.max_len):
        self.df = pd.read_csv(path).dropna()
        self.tokenizer=tokenizer
        self.max_token_len=max_token_len
        self.labelstrdicts={1:"ESTJ", 0:"INFP"}
        self.loc=i #EI at index 0 in mbti
    def __len__(self):
        return (len(self.df))
    def __getitem__(self, index):
        item=self.df.iloc[index]
        text=item["post"]
        t=item["type"]
        labels=self.str2label(t)
        try:
          tokens=self.tokenizer(text,return_tensors="pt", truncation=True, max_length=self.max_token_len, padding="max_length")
        except:
          print(text)
          quit()
        return {"input_ids": torch.squeeze(tokens.input_ids), "attention_mask":torch.squeeze(tokens.attention_mask), "labels":labels}
    def str2label(self, string):
        letter=string[self.loc]
        if letter in "ESTJ":
            return 1
        else:
            return 0
    def label2str(self, label):
        return self.labelstrdicts[label][self.loc]



In [7]:
#from transformers import BertTokenizer, BertForSequenceClassification
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
#tokenizer = BertTokenizer.from_pretrained(cfg.model)
tokenizer = DistilBertTokenizer.from_pretrained(cfg.model)
config = AutoConfig.from_pretrained(cfg.model)
#bertmodel= BertForSequenceClassification.from_pretrained(cfg.model, config=config)
distilbertmodel= DistilBertForSequenceClassification.from_pretrained(cfg.model, config=config)
#bertmodel = AutoModelForSequenceClassification.from_pretrained(model_path)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

In [8]:
for name, param in distilbertmodel.named_parameters():
  print(name)
  if name.startswith('distilbert.transformer.layer.5'):
    if name.startswith('distilbert.transformer.layer.5.atten'):
      param.requires_grad = False
      continue
    else:
      param.requires_grad = True
      continue
  else:
    if name.startswith('distilbert.transformer.layer.'):
      param.requires_grad = False
      continue
    else:
      param.requires_grad = True
      continue
  

distilbert.embeddings.word_embeddings.weight
distilbert.embeddings.position_embeddings.weight
distilbert.embeddings.LayerNorm.weight
distilbert.embeddings.LayerNorm.bias
distilbert.transformer.layer.0.attention.q_lin.weight
distilbert.transformer.layer.0.attention.q_lin.bias
distilbert.transformer.layer.0.attention.k_lin.weight
distilbert.transformer.layer.0.attention.k_lin.bias
distilbert.transformer.layer.0.attention.v_lin.weight
distilbert.transformer.layer.0.attention.v_lin.bias
distilbert.transformer.layer.0.attention.out_lin.weight
distilbert.transformer.layer.0.attention.out_lin.bias
distilbert.transformer.layer.0.sa_layer_norm.weight
distilbert.transformer.layer.0.sa_layer_norm.bias
distilbert.transformer.layer.0.ffn.lin1.weight
distilbert.transformer.layer.0.ffn.lin1.bias
distilbert.transformer.layer.0.ffn.lin2.weight
distilbert.transformer.layer.0.ffn.lin2.bias
distilbert.transformer.layer.0.output_layer_norm.weight
distilbert.transformer.layer.0.output_layer_norm.bias
distil

In [12]:
#path=foldername+"dataset2.csv"
path=foldername+"balanced_FT_dataset.csv"
dataset_EI=Ds(path, tokenizer, i=0)

#dataset_SN=Ds_SN(path, tokenizer)
#dl_SN=getdl(dataset_SN)

#dataset_TF=Ds_TF(path, tokenizer)
#dl_TF=getdl(dataset_TF)

#dataset_JP=Ds_JP(path, tokenizer)
#dl_JP=getdl(dataset_JP)

In [13]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
device

'cuda'

In [16]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 3.2726])).to(device)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1)).to(device)
        return (loss, outputs) if return_outputs else loss

In [17]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=foldername+"test_trainer", 
                                  evaluation_strategy="epoch", 
                                  num_train_epochs=30, 
                                  per_device_train_batch_size=32, 
                                  per_device_eval_batch_size=32,
                                  save_steps=1e5,
                                  weight_decay=0.01,
                                  )

In [18]:
total_len=len(dataset_EI)
train_len=int(len(dataset_EI)*0.9)
val_len=int(total_len-train_len)
[train_ds, val_ds]=torch.utils.data.random_split(dataset_EI, [train_len, val_len])
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

In [19]:
trainer = CustomTrainer(
    #model=bertmodel,
    model=distilbertmodel,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 327274
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 306840
  Number of trainable parameters = 29153282


Epoch,Training Loss,Validation Loss,Accuracy
1,0.657900,0.657510,0.644924
2,0.626000,0.658337,0.639259
3,0.586100,0.683859,0.650588
4,0.533100,0.794222,0.688703
5,0.474600,0.921243,0.681581
6,0.423900,1.113564,0.693186
7,0.372700,1.311903,0.697311
8,0.333600,1.451557,0.692718
9,0.297800,1.483840,0.703938
10,0.260300,1.695683,0.705175


***** Running Evaluation *****
  Num examples = 36364
  Batch size = 32
***** Running Evaluation *****
  Num examples = 36364
  Batch size = 32
***** Running Evaluation *****
  Num examples = 36364
  Batch size = 32
***** Running Evaluation *****
  Num examples = 36364
  Batch size = 32
***** Running Evaluation *****
  Num examples = 36364
  Batch size = 32
***** Running Evaluation *****
  Num examples = 36364
  Batch size = 32
***** Running Evaluation *****
  Num examples = 36364
  Batch size = 32
***** Running Evaluation *****
  Num examples = 36364
  Batch size = 32
***** Running Evaluation *****
  Num examples = 36364
  Batch size = 32
Saving model checkpoint to /content/drive/My Drive/2022Fall/CSCI3349/NLP Project/extractedfeaturewithbert/test_trainer/checkpoint-100000
Configuration saved in /content/drive/My Drive/2022Fall/CSCI3349/NLP Project/extractedfeaturewithbert/test_trainer/checkpoint-100000/config.json
Model weights saved in /content/drive/My Drive/2022Fall/CSCI3349/NLP P